# Rule Unit 작성 예시

본 문서는 1차년도 6월에 개발된 RuleUnit 클래스를 활용하여 룰 유닛을 작성하고 실행하는 과정을 설명합니다.

⚠︎ 주의사항: 코드를 변경하며 작업할 때는, 사본을 생성한 후 작업해주십시오.

## 룰 유닛 작성하기
이 부분에서는 기존의 IFC 파일에 종속적이었던 실행 흐름에서 벗어나, 새로이 정의한 RuleUnit 구조에 기반하여 룰에 포함된 항목을 작성하는 방법을 다룹니다.<br><br>
본 예시에서는 KDS 24 14 21 4.6.5.1(5) 를 룰 유닛으로 작성합니다.

먼저, Rule 작성에 필요한 프로그램을 불러오기 위해 Github에서 아래의 주소가 가리키는 파일 모음을 불러옵니다. (Github Repository를 clone합니다.)

In [ ]:
!git clone https://github.com/KU-HIAI/tomok.git

Cloning into 'tomok'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 175 (delta 54), reused 85 (delta 31), pack-reused 58
Receiving objects: 100% (175/175), 451.83 KiB | 4.11 MiB/s, done.
Resolving deltas: 100% (62/62), done.


필요한 파일을 불러온 다음에는 현재 작업 경로를 tomok의 파일이 있는 위치로 이동합니다.<br><br>
아래의 코드 블록을 실행하면, 작성자의 기본 작업 경로가 필요한 파일이 있는 위치로 이동됩니다.

In [ ]:
cd tomok

/content/tomok


## 룰 유닛 작성해보기
지금까지의 내용을 바탕으로 룰 유닛을 작성해봅시다.

새로운 룰 유닛을 작성하기 위해서는, 아래 코드 블럭에서 클래스 이름, 메타정보에 해당하는 변수, 실행 함수의 이름과 내용을 수정하면 됩니다.

In [ ]:
import math
from typing import List

import tomok

# 작성하는 룰에 맞게 클래스 이름 수정 (KDS241421_040609_02)
class KDS241421_040609_02 (tomok.RuleUnit):

    # 아래 클래스 멤버 변수에 할당되는 값들을 작성하는 룰에 맞게 수정
    priority = 1   # 건설기준 우선순위
    author = 'Jiwoo Won'  # 작성자명
    ref_code = 'KDS 24 14 21 4.6.9 (2)' # 건설기준문서
    ref_date = '2021-04-12'  # 디지털 건설문서 작성일
    doc_date = '2023-11-10'  # 건설기준문서->디지털 건설기준 변환 기준일
    title = '철근망의 두 인접한 철근 사이의 간격'    # 건설기준명

    #
    description = """
    콘크리트교 설계기준 (한계상태설계법)
    4. 설계
    4.6 부재 상세
    4.6.9 깊은 보
    (2)
    """
    # https://dillinger.io/ 표와 이미지 랜더링 확인 사이트
    # 이미지 링크 변환 사이트 https://www.somanet.xyz/2017/06/blog-post_21.html
    # 건설기준문서내용(text)
    content = """
    ####  4.6.9 깊은 보
    (2) 철근망의 두 인접한 철근 사이의 간격은 깊은 보 두께의 2배 또는 300mm 중 작은 값 이하여야 한다.

    """
    # 플로우차트(mermaid)
    flowchart = """
    flowchart TD
    subgraph Python_Class
    A["깊은 보"];
    B["KDS 24 14 21 4.6.9 (2)"];
    A ~~~ B
    end

		subgraph Variable_def;
		VarIn1[/입력변수:철근 사이의 간격/];
		VarIn2[/입력변수:깊은 보 두께/];


		VarIn1 & VarIn2
		end

		Python_Class ~~~ Variable_def
		Variable_def--->C


		C["철근 사이의 간격≤min(깊은 보 두께x2,300mm)"]

    """

    # 작성하는 룰에 맞게 함수 이름과 내용을 수정,
    @tomok.rule_method
    def Spacing_between_rebars(fIspbere,fIdebeth) ->bool:
        """철근망의 두 인접한 철근 사이의 간격
        Args:
             fIspbere (float): 철근 사이의 간격
             fIdebeth (float): 깊은 보 두께


        Returns:
            bool: 철근 사이의 간격이 콘크리트교 설계기준 (한계상태설계법) 4.6.9(2)의 기준을 만족하는지 여부
        """

        if fIspbere <= min( 2*fIdebeth, 300 ):
          return "Pass"
        else:
          return "Fail"

작성한 룰 유닛은 아래의 코드 블럭과 같이 생성하여, 작성자가 임의로 검증을 수행할 수 있습니다.

In [ ]:
my_RuleUnit = KDS241421_040609_02()

In [ ]:
fIspbere=280
fIdebeth=200

In [ ]:
Rule_Review_Result = my_RuleUnit.Spacing_between_rebars(fIspbere,fIdebeth)
# 해당건설기준 항목 의 값은 무엇인가?

print("RuleUnit Review Result: {}".format(Rule_Review_Result))

RuleUnit Review Result: Pass


<br><br>
아래의 코드를 통해, 룰 유닛의 content(건설기준 항목의 실제 내용)의 markdown 렌더링 결과를 확인할 수 있습니다.

In [ ]:
my_RuleUnit.render_markdown()


    ####  4.6.9 깊은 보
    (2) 철근망의 두 인접한 철근 사이의 간격은 깊은 보 두께의 2배 또는 300mm 중 작은 값 이하여야 한다.

    